<a href="https://colab.research.google.com/github/MatVitale6/UrbexProject/blob/master/Fine_Tuning_03_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install datasets

In [2]:
import pandas as pd
import json
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

In [3]:
from google.colab import files
uploaded = files.upload()

Saving RotatE.xlsx to RotatE (1).xlsx


In [4]:
#apro e divido il df
df = pd.read_excel(r"RotatE.xlsx")
print(df)

train_df, temp_df = train_test_split(df, test_size = 0.3, random_state=42) #forse dopo 0.4
val_df, test_df = train_test_split(temp_df, test_size = 0.5, random_state=42)

      Unnamed: 0    id    str predicted_label  ground_truth
0              0     1  [CLS]               O             O
1              1   248      R    B-MethodName  B-MethodName
2              2  3293     OT    I-MethodName  I-MethodName
3              3  8625    ATE    I-MethodName  I-MethodName
4              4    35      :               O             O
...          ...   ...    ...             ...           ...
5476        5476     0  [PAD]               O             O
5477        5477     0  [PAD]               O             O
5478        5478     0  [PAD]               O             O
5479        5479     0  [PAD]               O             O
5480        5480     0  [PAD]               O             O

[5481 rows x 5 columns]


In [20]:
print(type(train_df))
print(train_df.shape)

<class 'pandas.core.frame.DataFrame'>
(3836, 5)


In [5]:
#funzione che prende un df e lo trasforma in dataset dict
def dataframe_to_dict(dataframe):
    texts = []
    labels = []

    # Imposta il numero di righe per ogni gruppo
    batch_size = 10

    text_batch = []
    label_batch = []


    for _, row in dataframe.iterrows():
        #converto in stringhe perché mi data errore
        text = str(row['str'])
        label = str(row['ground_truth'])


        text_batch.append(text)
        label_batch.append(label)

        if len(text_batch) >= batch_size:
            texts.append(text_batch)
            labels.append(label_batch)

            text_batch = []
            label_batch = []


    if text_batch:
        texts.append(text_batch)
        labels.append(label_batch)


    dataset_dict = {
        'text': texts,
        'labels': labels
    }

    dataset = Dataset.from_dict(dataset_dict)

    return dataset

In [6]:
train_dataset = dataframe_to_dict(train_df)
val_dataset = dataframe_to_dict(val_df)
test_dataset = dataframe_to_dict(test_df)

dataset_dict = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset,
    "test": test_dataset
})

dataset_dict.save_to_disk("RotatE6_dataset")

Saving the dataset (0/1 shards):   0%|          | 0/384 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/83 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/83 [00:00<?, ? examples/s]

In [7]:
#a me servono int le label e quindi forziamole, speriamo bene

label_to_id = {
    'O': 0,
    'B-MetricName': 1,
    'I-MetricName': 2,
    'B-MethodName': 3,
    'I-MethodName': 4,
    'B-DatasetName': 5,
    'I-DatasetName': 6,
    'B-MetricValue': 7,
    'I-MetricValue': 8,
    'B-HyperparameterName': 9,
    'I-HyperparameterName': 10,
    'B-HyperparameterValue': 11,
    'I-HyperparameterValue': 12,
    'B-TaskName': 13,
    'I-TaskName': 14,
    'nan': 0
}

# Definisci una funzione per convertire le etichette in ID
def convert_labels_to_ids(example):
    # Estrai le etichette
    labels = example["labels"]
    # Mappa le etichette in ID utilizzando il tuo dizionario label_to_id
    labels_ids = [label_to_id[label] for label in labels]
    # Restituisci l'esempio con le etichette convertite in ID
    return {"text": example["text"], "labels": labels_ids}

# Applica la funzione di conversione a ciascun set di dati nel DatasetDict
for split in dataset_dict.keys():
    dataset_dict[split] = dataset_dict[split].map(convert_labels_to_ids)

print(dataset_dict["train"]["labels"])

Map:   0%|          | 0/384 [00:00<?, ? examples/s]

Map:   0%|          | 0/83 [00:00<?, ? examples/s]

Map:   0%|          | 0/83 [00:00<?, ? examples/s]

[[0, 0, 0, 0, 14, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 6, 3, 0, 0, 0, 0], [0, 0, 0, 9, 0, 0, 0, 0, 0, 14], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [3, 0, 0, 9, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 4, 0, 0, 0], [0, 14, 10, 0, 0, 0, 0, 0, 0, 14], [0, 3, 14, 6, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 4, 0, 4, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 14, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [6, 0, 10, 10, 0, 0, 0, 0, 0, 0], [0, 12, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 4, 0, 0, 0, 0, 0, 0, 0, 10], [0, 4, 0, 0, 5, 0, 0, 4, 9, 3], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 4, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 3, 0, 0, 0, 4], [0, 0, 10, 0, 0, 10, 0, 0, 6, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 3, 0, 3, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 9], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 3], [10, 0, 0, 0, 0, 0, 0, 0, 6, 0], [0, 0, 0, 4, 0, 0, 0, 0, 0, 0], [6, 0, 4, 0, 0, 0, 0, 0, 

In [8]:
label_names = ['O', 'B-MetricName', 'I-MetricName', 'B-MethodName', 'I-MethodName',
         'B-DatasetName', 'I-DatasetName','B-MetricValue', 'I-MetricValue',
    'B-HyperparameterName', 'I-HyperparameterName',
         'B-HyperparameterValue', 'I-HyperparameterValue',
        'B-TaskName','I-TaskName', 'nan']

In [11]:
pip install transformers[torch]

  Using cached accelerate-0.23.0-py3-none-any.whl (258 kB)
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.20.1
    Uninstalling accelerate-0.20.1:
      Successfully uninstalled accelerate-0.20.1


In [12]:
import transformers

from transformers import AutoModelForTokenClassification
from transformers import AutoTokenizer

model = AutoModelForTokenClassification.from_pretrained("sohamtiwari3120/scideberta-cs-tdm-pretrained-finetuned-ner")
tokenizer = AutoTokenizer.from_pretrained("sohamtiwari3120/scideberta-cs-tdm-pretrained-finetuned-ner")


In [14]:
#Get the values for input_ids, token_type_ids, attention_mask
def tokenize_adjust_labels(all_samples_per_split):
    tokenized_samples = tokenizer.batch_encode_plus(all_samples_per_split["text"], is_split_into_words=True)
    #tokenized_samples is not a datasets object so this alone won't work with Trainer API, hence map is used
    #so the new keys [input_ids, labels (after adjustment)]
    #can be added to the datasets dict for each train test validation split
    total_adjusted_labels = []
    print(len(tokenized_samples["input_ids"]))

    for k in range(0, len(tokenized_samples["input_ids"])):
        prev_wid = -1
        word_ids_list = tokenized_samples.word_ids(batch_index=k)
        existing_label_ids = all_samples_per_split["labels"][k]

        i = -1
        adjusted_label_ids = []


        for wid in word_ids_list:
            if(wid is None):
                adjusted_label_ids.append(-100)
            elif(wid!=prev_wid):
                i = i + 1
                adjusted_label_ids.append(existing_label_ids[i])
                prev_wid = wid
            else:
                label_name = label_names[existing_label_ids[i]]
                adjusted_label_ids.append(existing_label_ids[i])

        total_adjusted_labels.append(adjusted_label_ids)
    tokenized_samples["labels"] = total_adjusted_labels
    return tokenized_samples

tokenized_dataset = dataset_dict.map(tokenize_adjust_labels, batched=True)

Map:   0%|          | 0/384 [00:00<?, ? examples/s]

384


Map:   0%|          | 0/83 [00:00<?, ? examples/s]

83


Map:   0%|          | 0/83 [00:00<?, ? examples/s]

83


In [15]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [16]:
pip install seqeval

In [17]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
import numpy as np
from datasets import load_metric
metric = load_metric("seqeval")
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

  # Remove ignored index (special tokens)
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_names[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    flattened_results = {
        "overall_precision": results["overall_precision"],
        "overall_recall": results["overall_recall"],
        "overall_f1": results["overall_f1"],
        "overall_accuracy": results["overall_accuracy"],
    }
    for k in results.keys():
        if(k not in flattened_results.keys()):
            flattened_results[k+"_f1"]=results[k]["f1"]

    return flattened_results

<ipython-input-17-d390710b9622>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


In [18]:
import torch
import accelerate

training_args = TrainingArguments(
    output_dir="./fine_tune_Debert_output",
    evaluation_strategy="steps",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_steps = 1000,
    save_strategy='no'
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],   #prova con train, capisci bene questa funzione!
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

You're using a DebertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss


TrainOutput(global_step=48, training_loss=0.8432293732961019, metrics={'train_runtime': 234.0339, 'train_samples_per_second': 3.282, 'train_steps_per_second': 0.205, 'total_flos': 6839264380608.0, 'train_loss': 0.8432293732961019, 'epoch': 2.0})